###What is Fusion RAG?
- Fusion RAG stands for “Many Sources, One Answer.” It retrieves context from multiple retrievers (e.g., different documents or data types), then combines the retrieved results before sending them to an LLM for generation.

In [ ]:
# ===================== INSTALL DEPENDENCIES =====================
!pip install -q langchain sentence-transformers faiss-cpu pypdf groq langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [ ]:
# ================== IMPORTS ==================
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import (
    RunnableMap, RunnableLambda, RunnablePassthrough, RunnableSequence
)
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from google.colab import userdata

In [ ]:
# ================== LOAD LEGAL PDF ==================
loader = PyPDFLoader("/content/BharatKaSamvidhan.pdf")  # Replace with actual path
legal_docs = loader.load_and_split()

# ================== SIMULATED MULTILINGUAL DOCS ==================
multilingual_docs = [
    Document(page_content="नमस्ते! आप कानूनी सहायता हिंदी में प्राप्त कर सकते हैं। कृपया अपना प्रश्न पूछें।"),
    Document(page_content="You can ask questions in English or Hindi. The assistant will understand both."),
    Document(page_content="This assistant provides Indian constitutional legal guidance in multiple languages.")
]

In [ ]:
# ================== TEXT SPLITTER ==================
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
legal_split = splitter.split_documents(legal_docs)
multi_split = splitter.split_documents(multilingual_docs)

# ================== EMBEDDING MODELS ==================
legal_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
hindi_embedder = HuggingFaceEmbeddings(model_name="AkshitaS/bhasha-embed-v0")

/tmp/ipython-input-4-1902570644.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  legal_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mode

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# ================== VECTOR STORES ==================
legal_store = FAISS.from_documents(legal_split, legal_embedder)
multi_store = FAISS.from_documents(multi_split, hindi_embedder)

# ================== RETRIEVERS ==================
legal_retriever = legal_store.as_retriever(search_kwargs={"k": 3})
multi_retriever = multi_store.as_retriever(search_kwargs={"k": 2})

In [ ]:
# ================== LLM SETUP ==================
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=userdata.get("GROQ_API_KEY")
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fbab5efe210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fbaaeef0250>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# ================== MAIN PROMPT ==================
rag_prompt = PromptTemplate.from_template(
    "Using the following context, answer the question:\n\n{context}\n\nQuestion: {question}"
)

# ================== QUERY REWRITE PROMPT ==================
query_rewrite_prompt = ChatPromptTemplate.from_template(
    "You are a multilingual query optimizer. Convert the user query to English and make it more specific for a legal RAG system.\n\n"
    "Original Query: {original_query}\n\nRewritten Query:"
)

In [ ]:
# ================== QUERY BUILDER REWRITER CHAIN ==================
query_rewriter_chain = query_rewrite_prompt | llm | StrOutputParser()

In [ ]:
# ================== FUSION RETRIEVER ==================
fusion_retriever = RunnableLambda(
    lambda x: legal_retriever.get_relevant_documents(x["rewritten_query"]) +
              multi_retriever.get_relevant_documents(x["rewritten_query"])
)
fusion_retriever

RunnableLambda(...)

In [ ]:
# ================== MEMORY ==================
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(llm=llm, memory=memory)

/tmp/ipython-input-10-300296035.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
/tmp/ipython-input-10-300296035.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm, memory=memory)


In [ ]:
# ================== FINAL CHAIN ==================
fusion_rag_chain = (
    # Step 1: Rewrite query
    RunnableMap({
        "original_query": RunnablePassthrough()
    }) |
    RunnableMap({
        "rewritten_query": query_rewriter_chain
    }) |
    # Step 2: Fusion retrieval using rewritten query
    RunnableMap({
        "context": fusion_retriever,
        "question": lambda x: x["rewritten_query"]
    }) |
    # Step 3: Format prompt + generate
    rag_prompt |
    llm |
    StrOutputParser()
)
fusion_rag_chain

{
  original_query: RunnablePassthrough()
}
| {
    rewritten_query: ChatPromptTemplate(input_variables=['original_query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], input_types={}, partial_variables={}, template='You are a multilingual query optimizer. Convert the user query to English and make it more specific for a legal RAG system.\n\nOriginal Query: {original_query}\n\nRewritten Query:'), additional_kwargs={})])
                     | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fbab5efe210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fbaaeef0250>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))
                     | StrOutputParser()
  }
| {
    context: RunnableLambda(...),
    question: RunnableLambda(...)
  }
| PromptTemplate(input_variables=['context', 'question'], input_types={},

In [ ]:
# ================== RUN QUERY ==================
user_query = "क्या अनुच्छेद 21 जीवन के अधिकार को कवर करता है?"

# Add to memory (optional)
memory.chat_memory.add_user_message(user_query)

# Run chain
response = fusion_rag_chain.invoke(user_query)

# Save to memory
memory.chat_memory.add_ai_message(response)

# Show response
print("Answer:\n", response)

/tmp/ipython-input-9-3919814349.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  lambda x: legal_retriever.get_relevant_documents(x["rewritten_query"]) +


Answer:
 Article 21 of the Indian Constitution indeed guarantees the right to life and personal liberty. It states that "No person shall be deprived of his life or personal liberty except according to procedure established by law." This provision is a fundamental right that is protected by the Constitution and is considered essential for the dignity and well-being of individuals.

The implications and scope of Article 21 are far-reaching and have been interpreted by the courts to include a wide range of rights, such as:

1. Right to live with dignity: The Supreme Court has held that the right to life includes the right to live with dignity, which encompasses the right to a minimum standard of living, including access to food, water, shelter, and healthcare.
2. Right to personal liberty: The right to personal liberty includes the right to freedom from arbitrary detention, torture, and other forms of ill-treatment.
3. Right to privacy: The Supreme Court has recognized the right to privac

Legal Retriever	Extracts relevant text from the Constitution<br>
Multilingual Retriever	Supports Hindi/English queries<br>
Query Rewriter	Converts user queries to better structured retrieval prompts<br>
Fusion RAG	Combines both sources for richer answers<br>
Memory	Tracks prior Q&A for follow-up support<br>